In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))
# print((torch.cuda._get_nvml_device_index(0)))

In [3]:
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import ElasticsearchStore
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import multiprocessing
import os

from pathlib import Path
from langchain.prompts import load_prompt

In [4]:
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, ".."))
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "hyperclova", "hyperclova-seed-text-1.5b-q4-k-m.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "hyperclova", "hyperclova-seed-text-1.5b.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "kanana", "kanana-nano-2.1b-instruc-q4-k-m.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M", "llama-3.2-Korean-Bllossom-3B-gguf-Q4_K_M.gguf")

# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "Meta-Llama-3-8B-Instruct-GGUF", "Meta-Llama-3-8B-Instruct-Q4_K_M.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "llama-3.2-Korean-Bllossom-3B-GGUF", "llama-3.2-Korean-Bllossom-3B.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "EXAONE-3.5-2.4B-Instruct-GGUF", "EXAONE-3.5-2.4B-Instruct-BF16.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "EXAONE-3.5-2.4B-Instruct-GGUF", "EXAONE-3.5-2.4B-Instruct-Q5_K_M.gguf")


# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "TheBloke", "OpenHermes-2.5-Mistral-7B-GGUF", "openhermes-2.5-mistral-7b.Q3_K_M.gguf")
MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "TheBloke", "CapybaraHermes-2.5-Mistral-7B-GGUF", "capybarahermes-2.5-mistral-7b.Q6_K.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "unsloth", "Llama-3.1-8B-Instruct-GGUF", "Llama-3.1-8B-Instruct-UD-Q8_K_XL.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "unsloth", "Llama-3.1-8B-Instruct-GGUF", "Llama-3.1-8B-Instruct-Q3_K_M.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "unsloth", "Llama-3.1-8B-Instruct-GGUF", "Llama-3.1-8B-Instruct-UD-Q6_K_XL.gguf")
# MODEL_PATH = os.path.join(project_root, "ai_models", "base_models", "lmstudio-community", "Phi-4-mini-instruct-GGUF", "Phi-4-mini-instruct-Q3_K_L.gguf")


In [5]:
PROMPT_DIR = Path("prompts")
system_prompt = (PROMPT_DIR / "system" / "base_system_prompt_v0.4.txt").read_text(encoding="utf-8")
qa_prompt = load_prompt(str(PROMPT_DIR / "tasks" / "qa_prompt_v0.4.yaml"), encoding="utf-8")

condense_prompt = load_prompt(str(PROMPT_DIR / "follow" / "condense_question.yaml"), encoding="utf-8")

In [6]:
# HuggingFaceEmbeddings 클래스 deprecation 경고 발생
# LangChain 0.2.2에서 deprecated되어 1.0에서 제거 예정
# 해결: langchain-huggingface 패키지의 새 버전 사용 필요

embeddings = HuggingFaceEmbeddings(
    model_name="../ai_models/base_models/BGE-m3-ko",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

In [7]:
from langchain_elasticsearch import ElasticsearchStore

elastic_vector_search = ElasticsearchStore(
    es_url="http://127.0.0.1:9200",
    index_name="all_docs",
    embedding=embeddings,
    es_user="elastic",
    es_password="elastic",
)

In [8]:
# retriever = elastic_vector_search.as_retriever(
#     search_type="similarity",
#     search_kwargs={
#         "k": 20,
#         "field": "embedding",
#         "similarity": "cosine"
#     }
# )

retriever = elastic_vector_search.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,                    # 상위 5개 문서
        "num_candidates": 20,      # 후보 문서 수
        "field": "embedding",      # 벡터 필드
        "similarity": "cosine",    # 코사인 유사도
        "index_options": {
            "type": "int8_hnsw",   # HNSW 알고리즘
            "m": 16,               # 그래프 연결 수
            "ef_construction": 100  # 정확도 파라미터
        }
    }
)

In [9]:
# from elasticsearch import Elasticsearch

# es = Elasticsearch("http://127.0.0.1:9200", basic_auth=("elastic", "elastic"))

# try:
#     es.indices.update_aliases(body={
#         "actions": [
#             {"add": {"index": "pdf", "alias": "all_docs"}},
#             {"add": {"index": "korquad", "alias": "all_docs"}}
#         ]
#     })
# except Exception as e:
#     print(f"Error updating aliases: {e}")


In [10]:

# # 모든 인덱스 조회
# indices = es.indices.get_alias().keys()
# print("Available indices:", list(indices))

# # 각 인덱스의 문서 수 확인
# for index in indices:
#     count = es.count(index=index)
#     print(f"\nIndex: {index}")
#     print(f"Document count: {count['count']}")
    
#     # 인덱스 매핑 정보 조회
#     mapping = es.indices.get_mapping(index=index)
#     print(f"Mapping:", mapping[index]['mappings'])


In [11]:
# faiss_index_directory = "./faiss_index_directory"
# vectorstore = FAISS.load_local(faiss_index_directory, embeddings, allow_dangerous_deserialization=True)
# retriever = vectorstore.as_retriever(    
#     search_type="similarity_score_threshold", 
#     search_kwargs={"score_threshold": 0.5, "k": 3}
# )

In [12]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path=MODEL_PATH,          # 모델 파일 경로
    temperature=0.6,                # 생성의 무작위성/창의성 조절 (0-1, 높을수록 창의적)
    top_p=0.9,                      # 누적 확률 임계값 (다양성과 품질의 균형)
    max_tokens=512,                 # 생성할 최대 토큰 수
    repeat_penalty=1.3,             # 반복 표현 억제를 위한 페널티 계수
   # stop=["\n###","<<END>>"],     # 생성 중단 토큰 (주석 처리됨)
    presence_penalty=0.2,           # 이미 등장한 토큰 재사용 억제 강도
    frequency_penalty=0.3,          # 자주 등장하는 토큰 사용 억제 강도
    top_k=45,                      # 다음 토큰 선택시 고려할 상위 토큰 수
    callback_manager=callback_manager, # 콜백 관리자 (스트리밍 출력용)
    verbose=True,                   # 상세 로그 출력 여부
    n_ctx=4048,                    # 컨텍스트 윈도우 크기 (토큰 수)
    n_gpu_layers=-1,               # GPU로 처리할 레이어 수 (-1은 전체)
    n_batch=512,                   # 배치 처리 크기
    device="cuda",                 # 실행 디바이스 (GPU 사용)
    f16_kv=True                    # FP16 형식으로 key/value 캐시 저장 (메모리 최적화)
)

# llm = LlamaCpp(
#     model_path=MODEL_PATH,
#     temperature=0.6,
#     max_tokens=512,
#     top_p=1,
#     callback_manager=callback_manager,
#     verbose=True,
#     n_ctx=2048,  # 컨텍스트 길이
#     n_gpu_layers=-1,  # 모든 레이어를 GPU에서 실행 (-1은 전체 레이어)
#     n_batch=512,  # GPU 배치 크기
#     # n_threads=multiprocessing.cpu_count() - 1,
#     device="cuda",  # GPU 사용 설정
#     f16_kv=True  # GPU 메모리 최적화를 위한 FP16 사용
# )

f:\chat_test\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3607: UserWarning: WARNING! presence_penalty is not default parameter.
                presence_penalty was transferred to model_kwargs.
                Please confirm that presence_penalty is what you intended.
  if await self.run_code(code, result, async_=asy):
f:\chat_test\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3607: UserWarning: WARNING! frequency_penalty is not default parameter.
                frequency_penalty was transferred to model_kwargs.
                Please confirm that frequency_penalty is what you intended.
  if await self.run_code(code, result, async_=asy):
f:\chat_test\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3607: UserWarning: WARNING! device is not default parameter.
                device was transferred to model_kwargs.
                Please confirm that device is what you intended.
  if await self.run_code(code, result, async_=asy):
llama_model_loader: loade

In [13]:
merged_template_str = f"""{system_prompt}

{qa_prompt.template}"""   # ← .template 속성이 ‘본문 문자열’

# ④ 최종 PromptTemplate 생성
prompt_template = PromptTemplate(
    template=merged_template_str,
    input_variables=qa_prompt.input_variables   # ["context", "question", "chat_history"]
)

In [14]:
from langchain.chains import ConversationalRetrievalChain

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    condense_question_prompt=load_prompt(PROMPT_DIR / "follow" / "condense_question.yaml"),
    combine_docs_chain_kwargs={"prompt": prompt_template}
)


In [15]:
chat_history = [] # 첫 질문이면 빈 리스트

In [16]:
query = "초거대 AI에 대해 알려줘"

In [17]:
response = qa_chain.invoke(
    {"question": query, "chat_history": chat_history}
)

answer = response["answer"]            # or .get("answer")
source_docs = response.get("source_documents", [])


초거대 AI, 즉 Large Language Model는 자연어처리와 관련된 AI모델 중에서 특히 텍스트를 입력받아 문장을 생성하고 이해하며 추천할 수 있도록 구축한 것으로, 최근에는 ChatGPT와 같은 AI기반의 대화형 텍스트봇(chatbot)을 개발하거나 자동완성과 관련된 서비스를 포함해서 다양한 분야에 활용되고 있습니다.

llama_perf_context_print:        load time =   40201.68 ms
llama_perf_context_print: prompt eval time =   40201.21 ms /   387 tokens (  103.88 ms per token,     9.63 tokens per second)
llama_perf_context_print:        eval time =   29580.07 ms /   162 runs   (  182.59 ms per token,     5.48 tokens per second)
llama_perf_context_print:       total time =   70096.11 ms /   549 tokens


In [18]:
print(answer)

초거대 AI, 즉 Large Language Model는 자연어처리와 관련된 AI모델 중에서 특히 텍스트를 입력받아 문장을 생성하고 이해하며 추천할 수 있도록 구축한 것으로, 최근에는 ChatGPT와 같은 AI기반의 대화형 텍스트봇(chatbot)을 개발하거나 자동완성과 관련된 서비스를 포함해서 다양한 분야에 활용되고 있습니다.


In [19]:
print(source_docs)

[]


In [20]:
print(chat_history)

[]


In [21]:
# vectorstore.similarity_search("공공부문 ai 도입 현황 알려줘", k=20)